In [4]:
# import the necessary packages
import numpy as np
import cv2
import math
import os
from datetime import date


if not os.path.exists("dataset"):
    os.makedirs("dataset")

# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
#classes=['Person','Car']
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    #person_ind = [i for i, cls in enumerate(net.classes) if cls == 'person'][0]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
#colors = np.random.uniform(0, 255, size=(len(classes), 3))


cap = cv2.VideoCapture('aa.avi')

count =0
count_pic=0
count_off=0

while cap.isOpened():
    # Capture frame-by-frame
    if count_pic % 24==0:    
        today = date.today()
        path1="offenders\\"+str(today)
        if not os.path.exists(path1):
            os.makedirs(path1)
       
    
    ret, img = cap.read()

    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    
    count_ppl=0
    l=[]
    l=[]
    lf=[]
    j=1
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            #color = colors[i]
            if label=='person':
                cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0), 2)
                l=[]
                l.append(x)
                l.append(y)
                lf.append(l)
                count_ppl+=1
            
            #cv2.circle(img, (center_x, center_y), 10, (0,255,0), 3)
           # cv2.putText(img, label, (x, y + 30), font, 3, color, 1)
            s=str(j)
            j+=1
            
    close_person=""
    off=0
    for i in range(len(lf)):
        for j in range(i+1,len(lf)):
            d=math.sqrt( ((lf[j][1]-lf[i][1])**2)+((lf[j][0]-lf[i][0])**2) )
            if d<60:
                close_person+="Person "+str(i+1)+" and Person "+str(j+1)+" ; "
                img = cv2.line(img, (lf[i][0]+15,lf[i][1]+35), (lf[j][0]+15,lf[j][1]+35), (0,0,255), 2)
                off+=1

            
    count+=1    
    if count>=5:
        print("FRAME "+str(count_pic)+"    People Count : "+str(count_ppl)+"   RL : "+str(off))
        cv2.imwrite('dataset\\img'+str(count_pic)+'.png',img)
        count_pic+=1
        if off>1:
            #cv2.imwrite(path1+'\\'+gesture_foldername+str(count)+'.png',roi)
            cv2.imwrite(path1+'\\'+str(count_off)+'.png',img)
            count_off+=1
            
    
    if count_ppl>=6 and count>=5:
        a="HIGH ALERT "+str(count_ppl)+" people in your area!"
        print(a)    
    
    
    if count>=5:
        count=0 
        off=0
    
    
    cv2.imshow('frame',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

FRAME 0    People Count : 3   RL : 0
FRAME 1    People Count : 3   RL : 0
FRAME 2    People Count : 4   RL : 0
FRAME 3    People Count : 4   RL : 1
FRAME 4    People Count : 5   RL : 4
FRAME 5    People Count : 3   RL : 1
FRAME 6    People Count : 3   RL : 1
FRAME 7    People Count : 6   RL : 3
HIGH ALERT 6 people in your area!
FRAME 8    People Count : 5   RL : 1
FRAME 9    People Count : 5   RL : 1
FRAME 10    People Count : 4   RL : 1
FRAME 11    People Count : 4   RL : 1
FRAME 12    People Count : 5   RL : 2
FRAME 13    People Count : 5   RL : 1
FRAME 14    People Count : 5   RL : 1
FRAME 15    People Count : 6   RL : 1
HIGH ALERT 6 people in your area!
FRAME 16    People Count : 6   RL : 1
HIGH ALERT 6 people in your area!
FRAME 17    People Count : 6   RL : 1
HIGH ALERT 6 people in your area!


-1